## ■ 데이터별 사용하는 알고리즘 분류
    수치형 데이터   ->   명목형 데이터   ->   숫자와 문자가 섞여있는 경우
    kNN 알고리즘         나이브베이즈               의사결정트리

In [1]:
# 예제: 구매여부에 가장 큰 영향을 준 column을 찾기위해 정보획득량을 구하시오
skin <- read.csv('skin.csv',head=T, stringsAsFactors = T)
str(skin)
head(skin)

'data.frame':	30 obs. of  7 variables:
 $ cust_no    : int  1 2 3 4 5 6 7 8 9 10 ...
 $ gender     : Factor w/ 2 levels "남","여": 1 2 2 2 2 2 2 2 2 1 ...
 $ age        : Factor w/ 3 levels "20대","30대",..: 2 1 1 3 2 2 1 1 2 3 ...
 $ job        : Factor w/ 2 levels "NO","YES": 1 2 2 1 1 1 1 1 2 2 ...
 $ marry      : Factor w/ 2 levels "NO","YES": 2 2 2 1 2 1 2 2 2 1 ...
 $ car        : Factor w/ 2 levels "NO","YES": 1 2 1 1 1 2 1 2 1 2 ...
 $ cupon_react: Factor w/ 2 levels "NO","YES": 1 1 1 1 1 1 1 2 2 1 ...


cust_no,gender,age,job,marry,car,cupon_react
1,남,30대,NO,YES,NO,NO
2,여,20대,YES,YES,YES,NO
3,여,20대,YES,YES,NO,NO
4,여,40대,NO,NO,NO,NO
5,여,30대,NO,YES,NO,NO
6,여,30대,NO,NO,YES,NO


In [2]:
install.packages('FSelector')
library(FSelector)

Installing package into 'C:/Users/knitwill/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
also installing the dependency 'entropy'



package 'entropy' successfully unpacked and MD5 sums checked
package 'FSelector' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\knitwill\AppData\Local\Temp\RtmpYBFqWl\downloaded_packages


Warning message:
"package 'FSelector' was built under R version 3.6.3"

In [3]:
weights <- information.gain(cupon_react~., skin, unit='log2')
print(weights)

        attr_importance
cust_no     0.000000000
gender      0.116295990
age         0.028169964
job         0.019819441
marry       0.323650198
car         0.008690515


### ※ 문제239. 지방간을 일으키는 원인중에 가장 큰 영향력있는 요인이 무엇인지 정보획득량을 구해서 알아내시오
    데이터 fatliver2.csv

In [4]:
fat <- read.csv('fatliver2.csv',head=T,stringsAsFactors = T)
str(fat)

rs <- information.gain(FATLIVER~., fat, unit='log2')
print(rs)

'data.frame':	510 obs. of  5 variables:
 $ AGE     : Factor w/ 8 levels "10대","20대",..: 4 5 5 3 6 4 2 3 3 3 ...
 $ GENDER  : Factor w/ 2 levels "남자","여자": 1 2 2 1 2 2 1 1 1 2 ...
 $ DRINK   : Factor w/ 2 levels "음주많음","음주적음": 1 2 2 1 2 2 2 1 2 2 ...
 $ SMOKING : Factor w/ 2 levels "금연","흡연": 1 1 1 2 1 1 1 1 2 1 ...
 $ FATLIVER: Factor w/ 2 levels "no","yes": 2 1 1 1 1 1 2 2 1 1 ...
        attr_importance
AGE         0.032256902
GENDER      0.028650604
DRINK       0.012189492
SMOKING     0.009812076


In [5]:
# 1. 의사결정 패키지인 C50 패키지를 설치한다.
install.packages("C50")
library(C50)

Installing package into 'C:/Users/knitwill/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
also installing the dependencies 'libcoin', 'mvtnorm', 'Formula', 'inum', 'partykit', 'Cubist'



package 'libcoin' successfully unpacked and MD5 sums checked
package 'mvtnorm' successfully unpacked and MD5 sums checked
package 'Formula' successfully unpacked and MD5 sums checked
package 'inum' successfully unpacked and MD5 sums checked
package 'partykit' successfully unpacked and MD5 sums checked
package 'Cubist' successfully unpacked and MD5 sums checked
package 'C50' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\knitwill\AppData\Local\Temp\RtmpYBFqWl\downloaded_packages


Warning message:
"package 'C50' was built under R version 3.6.3"

In [6]:
# 2. 백화점 화장품 고객 데이터를 로드하고 shuffle 한다.
skin <- read.csv("skin.csv", header=T )
nrow(skin)

skin_real_test_cust <- skin[30,  ] # 모델 생성 후 정확도를 올린 후 최종적으로 모델이 잘 맞추는지 확인하기 위해 한 건 제외
skin2 <-  skin[ 1:29, ] 
nrow(skin2) 

skin_real_test_cust
#cust_no gender age job marry car cupon_react
#30      30 female  40 YES   YES  NO         YES

skin2 <- skin2[ , -1] # 고객번호를 제외시킨다. 

set.seed(11)
skin2_shuffle <- skin2[sample(nrow(skin2)),    ]  # shuffle 시킴 

[1] 30

[1] 29

,cust_no,gender,age,job,marry,car,cupon_react
30,30,여,40대,YES,YES,NO,YES


In [7]:
# 3. 화장품 고객 데이터를 7대 3로 train 과 test 로 나눈다.
train_num <-  round(0.7 * nrow(skin2_shuffle), 0) 
skin2_train <- skin2_shuffle[1:train_num,  ]  
skin2_test  <- skin2_shuffle[(train_num+1) : nrow(skin2_shuffle), ] 
nrow(skin2_train)  # 20
nrow(skin2_test)   #  9 

[1] 20

[1] 9

In [8]:
# 4. C50 패키지를 이용해서 분류 모델을 생성한다. 
library(C50)
skin_model <- C5.0(skin2_train[  , -6],  skin2_train$cupon_react )  
#                       ↑                        ↑
#               라벨을 뺀 train 전체 data    train 데이터의 라벨 

c50 code called exit with value 1


In [9]:
# 5. 위에서 만든 skin_model 를 이용해서 테스테 데이터의 라벨을 예측하시오!
skin2_result  <- predict( skin_model , skin2_test[  , -6])
#                                              ↑
#                                   라벨을 뺀 테스트 데이터 전체 

ERROR: Error in predict.C5.0(skin_model, skin2_test[, -6]): either a tree or rules must be provided


In [ ]:
# 6. 이원 교차표로 결과를 확인하시오 !
library(gmodels)
CrossTable( skin2_test[  , 6],  skin2_result ) 

### ※문제. 위의 의사결정트리의 성능을 높이시오 !